In [16]:
import tkinter
from tkinter import *
from tkinter import ttk
import pandas as pd # type: ignore
import numpy as np # type: ignore
from sklearn.metrics.pairwise import cosine_similarity # type: ignore
from scipy import sparse # type: ignore
from sklearn.metrics.pairwise import linear_kernel # type: ignore
from sklearn.feature_extraction.text import TfidfVectorizer # type: ignore


In [17]:
def get_dataframe_ratings_base(text):
    """
    đọc file base của movilens, lưu thành dataframe với 3 cột user id, item id, rating
    """
    r_cols = ['user_id', 'item_id', 'rating']
    ratings = pd.read_csv(text, sep='\t', names=r_cols, encoding='latin-1')
    Y_data = ratings.values
    return Y_data


def get_name_movie(text):
    """
    lấy danh sách tên phim theo file u.item của movielens
    """
    r_cols = ['name', 'year', 'imdb', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19','20','21','22']
    list_movie = pd.read_csv(text, sep='|', names=r_cols, encoding='latin-1')
    list_name_movie = list_movie['name'].values
    return list_name_movie


def get_year_movie(text):
    """
    lấy danh sách năm sản xuất phim theo file u.item của movielens
    """
    r_cols = ['name', 'year', 'imdb', '3','4', '5', '6','7', '8', '9','10', '11', '12','13', '14', '15','16','17', '18', '19','20','21','22']
    list_movie = pd.read_csv(text, sep='|', names=r_cols, encoding='latin-1')
    list_year_movie = list_movie['year'].values
    return list_year_movie


def get_dataframe_movies_csv(text):
    """
    đọc file csv của movilens, lưu thành dataframe với 3 cột user id, title, genres
    """
    movie_cols = ['movie_id', 'title', 'genres']
    movies = pd.read_csv(text, sep=',', names=movie_cols, encoding='latin-1')
    return movies


In [18]:
class CF(object):
    """
    class Collaborative Filtering, hệ thống đề xuất dựa trên sự tương đồng
    giữa các users với nhau, giữa các items với nhau
    """

    def __init__(self, data_matrix, k, dist_func=cosine_similarity, uuCF=1):
        """
        Khởi tạo CF với các tham số đầu vào:
            data_matrix: ma trận Utility, gồm 3 cột, mỗi cột gồm 3 số liệu: user_id, item_id, rating.
            k: số lượng láng giềng lựa chọn để dự đoán rating.
            uuCF: Nếu sử dụng uuCF thì uuCF = 1 , ngược lại uuCF = 0. Tham số nhận giá trị mặc định là 1.
            dist_f: Hàm khoảng cách, ở đây sử dụng hàm cosine_similarity của klearn.
            limit: Số lượng items gợi ý cho mỗi user. Mặc định bằng 10.
        """
        self.uuCF = uuCF  # user-user (1) or item-item (0) CF
        self.Y_data = data_matrix if uuCF else data_matrix[:, [1, 0, 2]]
        self.k = k
        self.dist_func = dist_func
        self.Ybar_data = None
        # số lượng user và item, +1 vì mảng bắt đầu từ 0
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1

    def add(self, new_data):
        """
        Cập nhật Y_data khi có lượt rating mới.
        """
        self.Y_data = np.concatenate((self.Y_data, new_data), axis=0)

    def normalize_matrix(self):
        """
        Tính similarity giữa các items bằng cách tính trung bình cộng ratings giữa các items.
        Sau đó thực hiện chuẩn hóa bằng cách trừ các ratings đã biết của item cho trung bình cộng
        ratings tương ứng của item đó, đồng thời thay các ratings chưa biết bằng 0.
        """
        users = self.Y_data[:, 0]
        self.Ybar_data = self.Y_data.copy()
        self.mu = np.zeros((self.n_users,))
        for n in range(self.n_users):
            ids = np.where(users == n)[0].astype(np.int32)
            item_ids = self.Y_data[ids, 1]
            ratings = self.Y_data[ids, 2]
            # take mean
            m = np.mean(ratings)
            if np.isnan(m):
                m = 0  # để tránh mảng trống và nan value
            self.mu[n] = m
            # chuẩn hóa
            self.Ybar_data[ids, 2] = ratings - self.mu[n]
        self.Ybar = sparse.coo_matrix(
            (self.Ybar_data[:, 2], (self.Ybar_data[:, 1], self.Ybar_data[:, 0])),
            (self.n_items, self.n_users),
        )
        self.Ybar = self.Ybar.tocsr()

    def similarity(self):
        """
        Tính độ tương đồng giữa các user và các item
        """
        eps = 1e-6
        self.S = self.dist_func(self.Ybar.T, self.Ybar.T)

    def refresh(self):
        """
        Chuẩn hóa dữ liệu và tính toán lại ma trận similarity. (sau khi một số xếp hạng được thêm vào).
        """
        self.normalize_matrix()
        self.similarity()

    def fit(self):
        self.refresh()

    def __pred(self, u, i, normalized=1):
        """
        Dự đoán ra ratings của các users với mỗi items.
        """
        # tìm tất cả user đã rate item i
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
        users_rated_i = (self.Y_data[ids, 0]).astype(np.int32)
        sim = self.S[u, users_rated_i]
        a = np.argsort(sim)[-self.k :]
        nearest_s = sim[a]
        r = self.Ybar[i, users_rated_i[a]]
        if normalized:
            # cộng với 1e-8, để tránh chia cho 0
            return (r * nearest_s)[0] / (np.abs(nearest_s).sum() + 1e-8)

        return (r * nearest_s)[0] / (np.abs(nearest_s).sum() + 1e-8) + self.mu[u]

    def pred(self, u, i, normalized=1):
        """
        Xét xem phương pháp cần áp dùng là uuCF hay iiCF
        """
        if self.uuCF:
            return self.__pred(u, i, normalized)
        return self.__pred(i, u, normalized)

    def print_list_item(self):
        for i in range(self.n_items):
            print(i)

    def recommend(self, u):
        """
        Determine all items should be recommended for user u.
        The decision is made based on all i such that:
        self.pred(u, i) > 0. Suppose we are considering items which
        have not been rated by u yet.
        """
        ids = np.where(self.Y_data[:, 0] == u)[0]
        items_rated_by_u = self.Y_data[ids, 1].tolist()
        recommended_items = []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                rating = self.__pred(u, i)
                if rating > 0:
                    recommended_items.append(i)

        return recommended_items

    def recommend_top(self, u, top_x):
        """
        Determine top 10 items should be recommended for user u.
        . Suppose we are considering items which
        have not been rated by u yet.
        """
        ids = np.where(self.Y_data[:, 0] == u)[0]
        items_rated_by_u = self.Y_data[ids, 1].tolist()
        item = {"id": None, "similar": None}
        list_items = []

        def take_similar(elem):
            return elem["similar"]

        for i in range(self.n_items):
            if i not in items_rated_by_u:
                rating = self.__pred(u, i)
                item["id"] = i
                item["similar"] = rating
                list_items.append(item.copy())

        sorted_items = sorted(list_items, key=take_similar, reverse=True)
        sorted_items.pop(top_x)
        return sorted_items

    def print_recommendation(self):
        """
        print all items which should be recommended for each user
        """
        print("Recommendation: ")
        for u in range(self.n_users):
            recommended_items = self.recommend(u)
            if self.uuCF:
                print("Recommend item(s):", recommended_items, "for user", u)
            else:
                print("Recommend item", u, "for user(s) : ", recommended_items)

In [19]:
def tfidf_matrix(movies):
    """
        Dùng hàm "TfidfVectorizer" để chuẩn hóa "genres" với:
        + analyzer='word': chọn đơn vị trích xuất là word
        + ngram_range=(1, 1): mỗi lần trích xuất 1 word
        + min_df=0: tỉ lệ word không đọc được là 0
        Lúc này ma trận trả về với số dòng tương ứng với số lượng film và số cột tương ứng với số từ được tách ra từ "genres"
    """
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), min_df=1)
    new_tfidf_matrix = tf.fit_transform(movies['genres'])
    return new_tfidf_matrix


def cosine_sim(matrix):
    """
            Dùng hàm "linear_kernel" để tạo thành ma trận hình vuông với số hàng và số cột là số lượng film
             để tính toán điểm tương đồng giữa từng bộ phim với nhau
    """
    new_cosine_sim = linear_kernel(matrix, matrix)
    return new_cosine_sim


In [20]:
class CB(object):
    """
        Khởi tại dataframe "movies" với hàm "get_dataframe_movies_csv"
    """
    def __init__(self, movies_csv):
        self.movies = get_dataframe_movies_csv(movies_csv)
        self.tfidf_matrix = None
        self.cosine_sim = None

    def build_model(self):
        """
            Tách các giá trị của genres ở từng bộ phim đang được ngăn cách bởi '|'
        """
        self.movies['genres'] = self.movies['genres'].str.split('|')
        self.movies['genres'] = self.movies['genres'].fillna("").astype('str')
        self.tfidf_matrix = tfidf_matrix(self.movies)
        self.cosine_sim = cosine_sim(self.tfidf_matrix)

    def refresh(self):
        """
             Chuẩn hóa dữ liệu và tính toán lại ma trận
        """
        self.build_model()

    def fit(self):
        self.refresh()

    def genre_recommendations(self, title, top_x):
        """
            Xây dựng hàm trả về danh sách top film tương đồng theo tên film truyền vào:
            + Tham số truyền vào gồm "title" là tên film và "topX" là top film tương đồng cần lấy
            + Tạo ra list "sim_score" là danh sách điểm tương đồng với film truyền vào
            + Sắp xếp điểm tương đồng từ cao đến thấp
            + Trả về top danh sách tương đồng cao nhất theo giá trị "topX" truyền vào
        """
        titles = self.movies['title']
        indices = pd.Series(self.movies.index, index=self.movies['title'])
        idx = indices[title]
        sim_scores = list(enumerate(self.cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:top_x + 1]
        movie_indices = [i[0] for i in sim_scores]
        return sim_scores, titles.iloc[movie_indices].values

    def print_recommendations(self, text, top_x):
        """
            In ra top film tương đồng với film truyền vào
        """
        print(self.genre_recommendations(text, top_x))

In [21]:
def gui(cf_rs, cb_rs, list_name_movie, list_year_movie):
    def get_id_user():
        """
        Lấy id người dùng và trả về danh sách phim
        """
        tv.delete(*tv.get_children())
        x1 = entry1.get()
        list_movies = cf_rs.recommend_top(int(x1),200)
        for i in range(200):
            tv.insert(parent='', index=i, iid=i, text='',
                      values=('{0:.{1}f}'.format(list_movies[i]['similar'], 2),
                              list_name_movie[list_movies[i]['id']],
                              list_year_movie[list_movies[i]['id']]))

    root = Tk()
    root.title('Recommendation System')
    root.iconbitmap('./recommendation-system-master/assets/logo.ico')
    root.resizable(False, False)

    window_height = 450
    window_width = 500

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    x_coordinate = int((screen_width / 2) - (window_width / 2))
    y_coordinate = int((screen_height / 2) - (window_height / 2))
    root.geometry("{}x{}+{}+{}".format(window_width, window_height, x_coordinate, y_coordinate))
    root.deiconify()

    canvas1 = tkinter.Canvas(root, width=window_width, height=window_height)
    canvas1.pack()

    label1 = tkinter.Label(root, text='Welcome to Recommendation System')
    label1.config(font=('Segoe UI Semibold', 14))
    canvas1.create_window(250, 40, window=label1)

    label2 = tkinter.Label(root, text='Type your id here')
    label2.config(font=('Segoe UI Symbol', 10))
    canvas1.create_window(175, 95, window=label2)

    entry1 = tkinter.Entry(root)
    canvas1.create_window(300, 95, window=entry1)

    button1 = tkinter.Button(text='Suggest me', command=get_id_user, bg='brown', fg='white',
                             font=('helvetica', 9, 'bold'))
    canvas1.create_window(250, 135, window=button1)

    tv = ttk.Treeview(root)
    tv['columns'] = ("SIMILAR RATE", "MOVIE NAME", "RELEASE DATE")
    tv.column('#0', width=0, stretch=NO)
    tv.column('SIMILAR RATE', anchor=CENTER, width=90)
    tv.column('MOVIE NAME', anchor=W, width=230)
    tv.column('RELEASE DATE', anchor=CENTER, width=90)

    tv.heading('#0', text='', anchor=CENTER)
    tv.heading('SIMILAR RATE', text='SIMILAR RATE', anchor=CENTER)
    tv.heading('MOVIE NAME', text='MOVIE NAME', anchor=CENTER)
    tv.heading('RELEASE DATE', text='RELEASE DATE', anchor=CENTER)

    def on_double_click(e):
        """
        Trả về danh sách phim phù hợp với phim vừa chọn
        :param e:
        :return:
        """
        item = tv.selection()[0]
        new_window = Toplevel(root)
        new_window.title("Content-based RS")
        new_window.geometry("440x350")
        cb_canvas = tkinter.Canvas(new_window)
        cb_canvas.pack()

        cb_label = tkinter.Label(new_window, text="This is list movies similarity with \n" + tv.item(item)['values'][1])
        cb_label.config(font=('Segoe UI Semibold', 10))
        cb_canvas.create_window(200, 40, window=cb_label)

        cb_tv = ttk.Treeview(new_window)
        cb_tv['columns'] = ("SIMILAR RATE", "MOVIE NAME", "ID MOVIE")
        cb_tv.column('#0', width=0, stretch=NO)
        cb_tv.column('SIMILAR RATE', anchor=CENTER, width=80)
        cb_tv.column('MOVIE NAME', anchor=W, width=200)
        cb_tv.column('ID MOVIE', anchor=CENTER, width=70)

        cb_tv.heading('#0', text='', anchor=CENTER)
        cb_tv.heading('SIMILAR RATE', text='SIMILAR RATE', anchor=CENTER)
        cb_tv.heading('MOVIE NAME', text='MOVIE NAME', anchor=CENTER)
        cb_tv.heading('ID MOVIE', text='ID MOVIE', anchor=CENTER)

        list_similarity, list_movies_cb = cb_rs.genre_recommendations(tv.item(item)['values'][1], 200)
        for i in range(200):
            cb_tv.insert(parent='', index=i, iid=i, text='',
                         values=('{0:.{1}f}'.format(list_similarity[i][1], 2),
                                 list_movies_cb[i],
                                 list_similarity[i][0]))
        cb_canvas.create_window(190, 200, window=cb_tv)

    tv.bind("<Double-1>", on_double_click)

    canvas1.create_window(250, 290, window=tv)

    root.mainloop()


In [22]:
data_matrix = get_dataframe_ratings_base('./Data_Model_predict/ub.base')
cf_rs = CF(data_matrix, k=2, uuCF=1)
cf_rs.fit()

cb_rs = CB('./Data_Model_predict/movies.csv')
cb_rs.fit()

list_name_movie = get_name_movie('./Data_Model_predict/u.item')
list_year_movie = get_year_movie('./Data_Model_predict/u.item')
gui(cf_rs, cb_rs, list_name_movie, list_year_movie)



c:\Users\HELLO\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\HELLO\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [26]:
def get_recommendations(cf_rs, cb_rs, user_id, list_name_movie, list_year_movie):
    """
    Lấy id người dùng và trả về danh sách phim
    """
    # Get the list of movies recommended for the given user_id
    list_movies = cf_rs.recommend_top(int(user_id), 200)
    
    # Create a list to store the results
    results = []

    # Populate the results list with movie details
    for i in range(200):
        movie_id = list_movies[i]['id']
        similarity = '{0:.{1}f}'.format(list_movies[i]['similar'], 2)
        movie_name = list_name_movie[movie_id]
        release_year = list_year_movie[movie_id]
        results.append((similarity, movie_name, release_year))
    
    # Print the results
    for result in results:
        print(f"SIMILAR RATE: {result[0]}, MOVIE NAME: {result[1]}, RELEASE DATE: {result[2]}")

def get_content_based_recommendations(cb_rs, movie_name):
    """
    Trả về danh sách phim phù hợp với phim vừa chọn
    """
    # Get the list of similar movies based on content
    list_similarity, list_movies_cb = cb_rs.genre_recommendations(movie_name, 200)
    
    # Create a list to store the results
    results = []

    # Populate the results list with movie details
    for i in range(200):
        similarity = '{0:.{1}f}'.format(list_similarity[i][1], 2)
        cb_movie_name = list_movies_cb[i]
        movie_id = list_similarity[i][0]
        results.append((similarity, cb_movie_name, movie_id))
    
    # Print the results
    for result in results:
        print(f"SIMILAR RATE: {result[0]}, MOVIE NAME: {result[1]}, ID MOVIE: {result[2]}")

# Example usage:
# Assuming you have defined cf_rs, cb_rs, list_name_movie, and list_year_movie
user_id = 4  # Replace with actual user ID
get_recommendations(cf_rs, cb_rs, user_id, list_name_movie, list_year_movie)

# Assuming you have a movie name to get content-based recommendations for
movie_name = "Some Movie Name"  # Replace with actual movie name
get_content_based_recommendations(cb_rs, movie_name)


SIMILAR RATE: 1.00, MOVIE NAME: GoldenEye (1995), RELEASE DATE: 01-Jan-1995
SIMILAR RATE: 1.00, MOVIE NAME: Twelve Monkeys (1995), RELEASE DATE: 01-Jan-1995
SIMILAR RATE: 1.00, MOVIE NAME: Richard III (1995), RELEASE DATE: 22-Jan-1996
SIMILAR RATE: 1.00, MOVIE NAME: Mighty Aphrodite (1995), RELEASE DATE: 30-Oct-1995
SIMILAR RATE: 1.00, MOVIE NAME: Postino, Il (1994), RELEASE DATE: 01-Jan-1994
SIMILAR RATE: 1.00, MOVIE NAME: Mr. Holland's Opus (1995), RELEASE DATE: 29-Jan-1996
SIMILAR RATE: 1.00, MOVIE NAME: French Twist (Gazon maudit) (1995), RELEASE DATE: 01-Jan-1995
SIMILAR RATE: 1.00, MOVIE NAME: From Dusk Till Dawn (1996), RELEASE DATE: 05-Feb-1996
SIMILAR RATE: 1.00, MOVIE NAME: Angels and Insects (1995), RELEASE DATE: 01-Jan-1995
SIMILAR RATE: 1.00, MOVIE NAME: Desperado (1995), RELEASE DATE: 01-Jan-1995
SIMILAR RATE: 1.00, MOVIE NAME: Disclosure (1994), RELEASE DATE: 01-Jan-1994
SIMILAR RATE: 1.00, MOVIE NAME: Eat Drink Man Woman (1994), RELEASE DATE: 01-Jan-1994
SIMILAR RATE: 1

KeyError: 'Some Movie Name'